Pricing European Options Through Binomial Trees

Importing Necessary Libraries

In [271]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime as dt
from datetime import timedelta
import math
import matplotlib.pyplot as plt
import statistics

User Imputs

In [272]:
symbol = input("Enter symbol: ")
expirey = int(input("Enter expirey index: "))
strike = float(input("Enter strike price: "))
option_type = input("Enter type of option (call/put): ")
number_of_steps = int(input("Enter number of desired time steps"))

Obtaining Volatility For Underlying Stock From Historic Close Prices

In [273]:
def get_historical_volatility(symbol):
    TRADING_DAYS = 252
    
    # Get today's date
    end_date = dt.today().strftime('%Y-%m-%d')

    # Calculate the start date (6 months before today)
    start_date = (dt.today() - timedelta(days=365)).strftime('%Y-%m-%d')

    # Get historical data
    stock_data = yf.download(symbol, start=start_date, end=end_date)

    returns = np.log(stock_data['Close']/stock_data['Close'].shift(1)) # Log returns
    returns = returns.iloc[1:] # Remove empty first row
    daily_volatility = statistics.stdev(returns[1:])
    annualized_volatility = daily_volatility*np.sqrt(TRADING_DAYS)

    return annualized_volatility


In [280]:
get_historical_volatility(symbol)

[*********************100%%**********************]  1 of 1 completed


0.3043144577236233

Options Data Fetcher from yfinance

In [274]:
def get_data(symbol, expirey, strike, type_): 
    """
    Inputs the symbol of the underlying stock, the option expiration date, 
    with index 0 being the current week and increasing indices 
    resulting in later expiration dates as organized on Yahoo Finance, desired strike price,
    and the type of option.
    """
    Ticker = yf.Ticker(symbol)

    stock_price = Ticker.info['currentPrice'] # Individually get stock price

    FD_date = Ticker.options[expirey] # Options expirey time
    desired_columns_yFinance = ['contractSymbol', 'strike', 'lastPrice', 'bid', 'ask',
                                'change', 'volume', 'openInterest', 'impliedVolatility']

    FD = Ticker.option_chain(FD_date) # Full dataframe of options of a given expirey time
    try:
        if type_ == 'call':
            FD_calls = FD.calls[desired_columns_yFinance] # Dataframe of calls
            call_data = FD_calls[FD_calls['strike'] == strike] # Call option market data of given strike

            if not call_data.empty: # Given a valid strike representing a real option
                call_data = call_data.reset_index(drop=True)  # Reset index
                call_data['Underlying Stock Price'] = stock_price

            return call_data

        elif type_ == 'put':
            FD_puts = FD.puts[desired_columns_yFinance] # Dataframe of puts
            put_data = FD_puts[FD_puts['strike'] == strike] # Put option market data of given strike

            if not put_data.empty: # Given a valid strike representing a real option
                put_data = put_data.reset_index(drop=True)  # Reset index
                put_data['Underlying Stock Price'] = stock_price

            return put_data

        else:
            raise ValueError("type_ must be 'call' or 'put'")

    except pd.errors.EmptyDataError:
        print("No comparable option with given strike available")

Calling the Data Fetcher Given the User's Inputs

In [275]:
data = get_data(symbol, expirey, strike, option_type)
data

,contractSymbol,strike,lastPrice,bid,ask,change,volume,openInterest,impliedVolatility,Underlying Stock Price
0,MSFT231027P00315000,315.0,8.77,9.0,9.15,-2.25,127.0,948,0.311469,316.88


Finding Time Until Expiration

In [276]:
# Get expirey date
expire_date = dt.strptime(yf.Ticker(f'{symbol}').options[expirey], '%Y-%m-%d')

# Get today's date
today = dt.today()

# Calculate the time period in days
time_period_days = (expire_date - today).days

# Convert days to months
time_period_months = time_period_days / 30  # Assuming an average month length of 30 days

Extracting Required Data For Options Pricing

In [277]:
def extract_data_from_dataframe(data):
    S = data['Underlying Stock Price'].values[0]
    K = data['strike'].values[0]
    r = float(yf.Ticker("^IRX").info["previousClose"]) / 100 # 13 week US treasury bill
    T = time_period_months
    N = number_of_steps
    sigma = get_historical_volatility(symbol)
    div = float(yf.Ticker(symbol).info['dividendYield'])
    
    return S, K, r, T, N, sigma, div

S, K, r, T, N, sigma, div = extract_data_from_dataframe(data)

[*********************100%%**********************]  1 of 1 completed


Storing Attributes of Stock Option

In [278]:
class option(object):

    def __init__(self, S, K, r, T, N, sigma, div, option_type, ):
        self.S = S
        self.K = K
        self.r = r
        self.T = T
        self.N = max(1, N) # N must have at least 1 time step
        self.STs = None  # Declare the stock prices tree
        self.sigma = sigma # Volatility
        self.div = div # Dividend ield
        self.is_call = option_type == 'call' # Call or put

        # Computed Values
        self.dt = T/float(N)  # Single time step, in years
        self.df = math.exp(
            -(self.r-self.div) * self.dt)  # Discount factor
        

    """ Pricing Functions"""   
    def __setup_parameters__(self):
        """ Required calculations for the model """
        self.M = self.N + 1  # Number of terminal nodes of tree
        self.u = math.exp(self.sigma * math.sqrt(self.dt)) # Expected value in the up state
        self.d = math.exp(-self.sigma * math.sqrt(self.dt)) # Expected value in the down state
        self.qu = (math.exp((-self.r-self.div)*self.dt) -
                   self.d) / (self.u-self.d)
        self.qd = 1-self.qu

    def _initialize_stock_price_tree_(self):
        # Initialize terminal price nodes to zeros
        self.STs = np.zeros(self.M)

        # Calculate expected stock prices for each node
        for i in range(self.M):
            self.STs[i] = self.S*(self.u**(self.N-i))*(self.d**i)

    def _initialize_payoffs_tree_(self):
        # Get payoffs when the option expires at terminal nodes
        payoffs = np.maximum(
            0, (self.STs-self.K) if self.is_call
            else(self.K-self.STs))

        return payoffs

    def _traverse_tree_(self, payoffs):
        # Starting from the time the option expires, traverse
        # backwards and calculate discounted payoffs at each node
        for i in range(self.N):
            payoffs = (payoffs[:-1] * self.qu +
                       payoffs[1:] * self.qd) * self.df

        return payoffs

    def __begin_tree_traversal__(self):
        payoffs = self._initialize_payoffs_tree_()
        return self._traverse_tree_(payoffs)

    def price(self):
        """ The pricing implementation """
        self.__setup_parameters__()
        self._initialize_stock_price_tree_()
        payoffs = self.__begin_tree_traversal__()
        
        return payoffs[0]  # Option value converges to first node (second code cell)
    

Determining The Implied Option's Price

In [279]:
our_option = option(S, K, r, T, N, sigma, div, option_type)

print(our_option.price())

38.853947983131334
